In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from data.transaction_data_generator import generate_transaction_data
from typing import List, Set, Dict, Tuple, Optional
from preparation.data_preparer import prepare_data
import plotly.express as px
from visualization.data_visualizer import visualize_data
from model.model_creator import create_model
from model.solution_parser import parse_solution
from pulp import *
from visualization.solution_data_visualizer import visualize_solution_data
import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
importance_factors = {"grocery":1, "fashion":2, "shopping":3, "travel":10, "rent": 1000, "unknown":10, "income":0}

In [4]:
transactions = generate_transaction_data()
prepare_data(transactions)
year = 2019
selected_year = transactions[transactions['year']==year].copy()

In [5]:
selected_year["importance"] = selected_year['type'].map(importance_factors)

In [6]:
text = widgets.HTML(
    value="<b>Budget Optimization</b>: Select inputs",
)

savings_slider = widgets.FloatSlider( min=0,
    max=0.8,
    step=0.05,
    description='Savings:',
    value=0.3)
grocery_slider = widgets.FloatSlider( min=0,
    max=500.0,
    step=5.00,
    description='Grocery:',
    value=50.0)
display(text, savings_slider,grocery_slider)

btn = widgets.Button(description='Optimize')
display(btn)

output = widgets.Output()
display(output)


def btn_eventhandler(obj):
   
    clear_output()
    display(text, savings_slider,grocery_slider,btn)

    savings = savings_slider.value
    grocery_per_week = grocery_slider.value
    model, variables = create_model(selected_year, savings, grocery_per_week)
    status = model.solve()

    if LpStatus[status] == 'Optimal':
        print("Optimization was successful")
    else:
        valid = widgets.Valid(
            value=False,
            description='Infeasible',
        )
        display(valid)
        return
    solution, objective = parse_solution(selected_year,variables,model)
    solved_data = selected_year.join(solution)
    remained_transactions_sum = sum(solved_data['solution']*solved_data['Debit'])
    print(f"Total expenses of proposed retroactive plan: {remained_transactions_sum} EUR")
    visualize_solution_data(solved_data, grocery_per_week)
btn.on_click(btn_eventhandler)



HTML(value='<b>Budget Optimization</b>: Select inputs')

FloatSlider(value=0.45, description='Savings:', max=0.8, step=0.05)

FloatSlider(value=0.0, description='Grocery:', max=500.0, step=5.0)

Button(description='Optimize', style=ButtonStyle())

Optimization was successful
Total expenses of proposed retroactive plan: -17536.489999999994 EUR
